In [258]:
import pandas as pd
import datetime as dt
import os
import numpy as np

In [295]:
Columns = pd.read_csv('Hourly_Column_Dictionary.csv') # Read column dictionary 
no_bound = (-np.inf,np.inf)
bounds = dict.fromkeys(Columns.columns[4:], no_bound)

def mask_outside(data, bounds):
    lower, upper = bounds
    data = data.mask(data< lower)
    data = data.mask(data> upper)
    return data

bounds['Li190'] = (-10, 2700)
bounds['Li200'] = (-10, 2700)
bounds['Air_Temp'] = (-40, 45)
bounds['5m_Temp'] = (-40, 45)
bounds['12m_Temp'] = (-40, 45)
bounds['Ground_Temp'] = (-40, 70)
bounds['Rel_Humidity'] = (0, 110)
bounds['Soil_Temp'] = (-40, 45)
bounds['10cm_Soil_Temp'] = (-40, 45)
bounds['20cm_Soil_Temp'] = (-40, 45)
bounds['50cm_Soil_Temp'] = (-40, 45)
bounds['Wind_Speed'] = (0,150)

bounds['Li190_Min'] = (-10, 2700)
bounds['Li200_Min'] = (-10, 2700)
bounds['Air_Temp_Min'] = (-40, 45)
bounds['5m_Temp_Min'] = (-40, 45)
bounds['12m_Temp_Min'] = (-40, 45)
bounds['Ground_Temp_Min'] = (-40, 70)
bounds['Rel_Humidity_Min'] = (0, 110)
bounds['Soil_Temp_Min'] = (-40, 45)
bounds['10cm_Soil_Temp_Min'] = (-40, 45)
bounds['20cm_Soil_Temp_Min'] = (-40, 45)
bounds['50cm_Soil_Temp_Min'] = (-40, 45)
bounds['Wind_Speed_Min'] = (0,150)

bounds['Li190_Max'] = (-10, 2700)
bounds['Li200_Max'] = (-10, 2700)
bounds['Air_Temp_Max'] = (-40, 45)
bounds['5m_Temp_Max'] = (-40, 45)
bounds['12m_Temp_Max'] = (-40, 45)
bounds['Ground_Temp_Max'] = (-40, 70)
bounds['Rel_Humidity_Max'] = (0, 110)
bounds['Soil_Temp_Max'] = (-40, 45)
bounds['10cm_Soil_Temp_Max'] = (-40, 45)
bounds['20cm_Soil_Temp_Max'] = (-40, 45)
bounds['50cm_Soil_Temp_Max'] = (-40, 45)
bounds['Wind_Speed_Max'] = (0,150)

bounds['Wind_Dir'] = (0,360)
bounds['Wind_Dir_STD'] = (0,np.inf)
bounds['Rain'] = (0,1825)

## Reformat Cass

In [296]:
Raw_Dir = r'Raw_Data/Cass_Hourly/' # Input 

Parsed_Dir = r'Formatted_Data/Cass_Hourly/' # Output
if not os.path.exists(Parsed_Dir):
    os.makedirs(Parsed_Dir)
    
Filtered_Dir = r'Filtered_Data/Cass_Hourly/' # Output
if not os.path.exists(Filtered_Dir):
    os.makedirs(Filtered_Dir)

ymin = 1997 # Start of series
ymax = 2020 # End of series
years = range(ymin,ymax+1) 


In [297]:
for y in years:
    print(y)
    # Try to fetch data for that year
    try:
        data = pd.read_csv(Raw_Dir+'Cass_'+str(y)+'_Hourly.csv')
    except:
        print('No data for '+str(y))
        continue
    
    # Create timeseries from time info
    time = pd.to_datetime(data['Year'], format='%Y') + pd.to_timedelta(data.Day-1, unit='D') + pd.to_timedelta(data.Time/100, unit='H')
    
    # Index dataset by timeseries
    data.index = time
    data = data[~data.index.duplicated(keep='first')] # Drop any duplicate rows
    
    # Create container for formatted raw data
    out_data = pd.DataFrame()
    out_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1,hour=1),dt.datetime(year=y+1,month=1,day=1),freq='H')
    
    # Create container for filtered data
    filtered_data = pd.DataFrame()
    filtered_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1,hour=1),dt.datetime(year=y+1,month=1,day=1),freq='H')
    
    # Attach data to out_data
    for C in Columns.columns[4:]:
        for c in Columns[C]:
            if c in list(data.columns):
                try:
                    # See if value can be interpreted as numeric
                    out_data[C] = pd.to_numeric(data[c], errors='coerce')
                    break
                except:
                    pass
    
    # Filter and attach data to filtered_data
    for C in Columns.columns[4:]:
        for c in Columns[C]:
            if c in list(data.columns):
                try:
                    # See if value can be interpreted as numeric
                    filtered_data[C] = mask_outside(pd.to_numeric(data[c], errors='coerce'), bounds[C])
                    break
                except:
                    pass
                
    out_data.index.names = ['Time'] # Set Index name to Time
    out_data.to_csv(Parsed_Dir+'Cass_'+str(y)+'_Hourly.csv') # Save to file
    
    filtered_data.index.names = ['Time'] # Set Index name to Time
    filtered_data.to_csv(Filtered_Dir+'Cass_'+str(y)+'_Hourly.csv') # Save to file

1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


## Reformat Chilton
Chilton data is in two large xlsx files, so needs slightly different treatment to Cass.

In [298]:
Raw_Dir = r'Raw_Data/Chilton_Hourly/' # Input 
Parsed_Dir = r'Formatted_Data/Chilton_Hourly/' # Output
if not os.path.exists(Parsed_Dir):
    os.makedirs(Parsed_Dir)

Filtered_Dir = r'Filtered_Data/Chilton_Hourly/' # Output
if not os.path.exists(Filtered_Dir):
    os.makedirs(Filtered_Dir)

files  = [f for f in os.listdir(Raw_Dir) if f.endswith('.xlsx')] # Find datafiles 
files.reverse()

In [299]:
for file in files:
    print('Reading '+file+'...')
    data = pd.read_excel(Raw_Dir+file)
    data.index = pd.to_datetime(data['date/time'],format='%Y-%m-%d %H:%M:%S')

    # Find timespan of dataset
    ymin = min(data.index).year
    ymax = max(data.index).year
    years = range(ymin,ymax+1) 
    data = data[~data.index.duplicated(keep='first')] # Drop any duplicate rows
    

    for y in years:
        print(y)
        # Create container for data
        out_data = pd.DataFrame()
        out_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1,hour=1),dt.datetime(year=y+1,month=1,day=1),freq='H')

        # Create container for filtered data
        filtered_data = pd.DataFrame()
        filtered_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1,hour=1),dt.datetime(year=y+1,month=1,day=1),freq='H')
    
        # Attach data to out_data
        for C in Columns.columns[4:]:
            for c in Columns[C]:
                if c in list(data.columns):
                    try:
                        # See if value can be interpreted as numeric
                        out_data[C] = pd.to_numeric(data[c], errors='coerce')
                        break
                    except:
                        pass
                    
        # Filter and attach data to filtered_data
        for C in Columns.columns[4:]:
            for c in Columns[C]:
                if c in list(data.columns):
                    try:
                        # See if value can be interpreted as numeric
                        filtered_data[C] = mask_outside(pd.to_numeric(data[c], errors='coerce'), bounds[C])
                        break
                    except:
                        pass

        out_data.index.names = ['Time'] # Set Index name to Time
        out_data.to_csv(Parsed_Dir+'Chilton_'+str(y)+'_Hourly.csv') # Save to file
        
        filtered_data.index.names = ['Time'] # Set Index name to Time
        filtered_data.to_csv(Filtered_Dir+'Chilton_'+str(y)+'_Hourly.csv') # Save to file

Reading Chilton_86_00_hourly.xlsx...
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
Reading Chilton_01_06_hourly.xlsx...
2001
2002
2003
2004
2005
2006


### Mend solar data from chilton

Chilton occasionally reads slightly less than 0 at night, this just retroactively passes a ammended timeseries to chilton with negative vals close to 0 set to 0

In [300]:
files = [f for f in os.listdir('c:/Users/callu/Documents/GEOG Project/All_Data/Formatted_Data/Chilton_Hourly//') if f.endswith('.csv')]

def mask_solar(data, bounds):
    lower_lower, lower, upper = bounds
    data = data.mask((data < lower) & (data > lower_lower), other=0)
    data = data.mask(data < lower_lower)
    data = data.mask(data > upper)
    return data

for f in files:
    raw = pd.read_csv('c:/Users/callu/Documents/GEOG Project/All_Data/Formatted_Data/Chilton_Hourly//'+f)
    filtered =  pd.read_csv('c:/Users/callu/Documents/GEOG Project/All_Data/Filtered_Data/Chilton_Hourly//'+f)
    raw.index = raw.Time
    filtered.index = raw.Time
    
    solar = mask_solar(raw.Li200, (-10,0,2700))
    
    filtered.Li200 = solar
    filtered.to_csv('c:/Users/callu/Documents/GEOG Project/All_Data/Filtered_Data/Chilton_Hourly//'+f,index=False)


## Format Cliflo Data
Similar to Chilton these are all found in big csv files

In [251]:
Raw_Dir = r'Raw_Data/Cliflo_Hourly/' # Input 
Parsed_Dir = r'Formatted_Data/' # Output
Filtered_Dir = r'Filtered_Data/' # Output

Columns = pd.read_csv('Hourly_Column_Dictionary.csv') # Read column dictionary 
files  = [f for f in os.listdir(Raw_Dir) if f.endswith('.txt')] # Find datafiles 


In [260]:
for file in files:
    print('Reading '+file+'...')
    data = pd.read_csv(Raw_Dir+file)
    data.index = pd.to_datetime(data['Date(NZST)'],format='%Y%m%d:%H%M')

    # Find timespan of dataset
    ymin = min(data.index).year
    ymax = max(data.index).year
    years = range(ymin,ymax+1) 
    data = data[~data.index.duplicated(keep='first')] # Drop any duplicate rows

    for y in years:
        print(y)
        # Create container for data
        out_data = pd.DataFrame()
        out_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1,hour=1),dt.datetime(year=y+1,month=1,day=1),freq='H')

        # Create container for filtered data
        filtered_data = pd.DataFrame()
        filtered_data.index = pd.date_range(dt.datetime(year=y,month=1,day=1,hour=1),dt.datetime(year=y+1,month=1,day=1),freq='H')
        
        # Attach data to out_data
        for C in Columns.columns[4:]:
            for c in Columns[C]:
                if c in list(data.columns):
                    try:
                        # See if value can be interpreted as numeric
                        out_data[C] = pd.to_numeric(data[c], errors='coerce')
                        break
                    except:
                        pass
            
        # Filter and attach data to filtered_data
        for C in Columns.columns[4:]:
            for c in Columns[C]:
                if c in list(data.columns):
                    try:
                        # See if value can be interpreted as numeric
                        filtered_data[C] = mask_outside(pd.to_numeric(data[c], errors='coerce'), bounds[C])
                        break
                    except:
                        pass
        out_data.index.names = ['Time'] # Set Index name to Time'
        filtered_data.index.names = ['Time'] # Set Index name to Time
        
        station = file[:-4]
        
        if not os.path.exists(Parsed_Dir+station+'_Hourly/'):
            os.makedirs(Parsed_Dir+station+'_Hourly/')
        if not os.path.exists(Filtered_Dir+station+'_Hourly/'):
            os.makedirs(Filtered_Dir+station+'_Hourly/')
        
        out_data.to_csv(Parsed_Dir+station+'_Hourly/'+station+'_'+str(y)+'_Hourly.csv') # Save to file
        out_data.to_csv(Filtered_Dir+station+'_Hourly/'+station+'_'+str(y)+'_Hourly.csv') # Save to file

Reading ArthursAWS_ScreenObs.txt...
2012
2013
Reading ArthursEWS.txt...
2005
2006
2007
2008
2009
2010
